In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("news.csv")
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
data.drop("Unnamed: 0", axis=1, inplace=True)

In [4]:
data["label"] = pd.get_dummies(data["label"], drop_first=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   uint8 
dtypes: object(2), uint8(1)
memory usage: 105.3+ KB


In [6]:
import re

In [7]:
new_text = []

In [8]:
for i in range(6335):
    text = re.sub('[^a-zA-Z]', ' ', data["text"][i])
    text = text.lower()
    new_text.append(text)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(new_text).toarray()
y = data["label"].values

In [10]:
X.shape

(6335, 65204)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

In [12]:
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train, y_train)

ypred = xgb.predict(X_test)

In [13]:
xgb.score(X_test, y_test)

0.9368686868686869

In [14]:
from sklearn.metrics import classification_report

In [15]:
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       769
           1       0.94      0.94      0.94       815

    accuracy                           0.94      1584
   macro avg       0.94      0.94      0.94      1584
weighted avg       0.94      0.94      0.94      1584

